In [2]:
import torch
import transformers
# from transformers import RobertaTokenizer, RobertaModel
from transformers import BertTokenizer, BertModel

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, download_url

import torch.nn.functional as F
import numpy as np
import re

/Users/mohit/opt/anaconda3/envs/exp_pytorch_only/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# print(torch_geometric.__version__)

In [4]:
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
bert_model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation = True, do_lower = True)

In [6]:
dialogue1 = {"label": ["Ketoconazole"], "disease": ["Dermatitis"], "dialog": ["Patient: There are small flesh-colored bumps on the finger joints, which are very itchy", "Doctor: Hello , How long has this been the case?", "Doctor: Did it grow last year?", "Doctor: Do you have both hands?", "Doctor: It may be tinea pedis or eczema", "Patient: It's been a few days, There is only one hand, at first only the index finger, now there are four fingers", "patient: what medicine?", "doctor: if there is only one hand, the possibility of tinea pedis is high, because eczema is generally symmetrical and long", " Doctor: It is recommended to spray locally [MASK]"], "original_dialog": ["Patient: There are small flesh-colored bumps on the finger joints, which are very itchy", "Doctor: Hello, how long has this been happening?", "Doctor : Have you grown in the past year?", "Doctor: Do you have both hands?", "Doctor: It may be tinea pedis or eczema", "Patient: It's been a few days and I only have one hand. At first I only had the index finger. Now All four fingers have", "Patient: What medicine to use?", "Doctor: If there is only one, it is very likely to consider tinea pedis, because eczema is generally symmetrical and long", "Doctor: It is recommended to spray locally [MASK]"] }
dialogue2 = {"label": ["mometasone furoate", "clarithromycin", "pudilan anti-inflammatory oral liquid", "licorice tablet"], "disease": ["rhinitis"], "dialog": ["patient : Can Xanthium rhinitis capsule be taken with Lanqin Oral Liquid?", "Doctor: Hello, do you feel any discomfort?", "Doctor: Congestion, runny nose and sore throat?", "Patient: stuffy nose and booger Physiological seawater can wash out the very sticky white mucus", "Patient: Itchy throat", "Patient: Slight cough", "Patient: Sinusitis", "Doctor: What medication did you use and how did it work", " Doctor: How long has this been the case?", "Doctor: Is the runny nose running down the throat?", "Patient: No backflow rhinitis has affected sleep for many years recently", "Patient: I used to take cephalosporin", "Patient: It can be relieved by taking it or not. Relapse slowly", "Doctor: Do you have sinusitis after sinus CT scan?", "Doctor: For the safety of your medication, do you have a history of drug allergies? Do you have poor liver and kidney function?", "Patient: Yes", "Patient: Sinusitis", "Patient: No allergies", "Patient: Normal liver and kidneys", "Patient: Only one of the two nostrils can pass through", "Patient: The pillow should be raised to the point of ventilation", " Doctor: According to your description, chronic sinusitis and chronic pharyngitis are likely to be considered. [MASK], [MASK], [MASK], [MASK]"], "original_dialog": ["Patient: Xanthium rhinitis capsule Can I take it with Lanqin Oral Liquid?", "Doctor: Hello, what's wrong with you?", "Doctor: Congestion, runny nose and sore throat?", "Patient: Congestion and excrement can be washed out by physiological seawater. Sticky white nasal discharge", "Patient: Itchy throat", "Patient: Cough a little", "Patient: Sinusitis", "Doctor: What medication has been used and how is it working", "Doctor: How long has this been happening? ", "Doctor: Does the nasal discharge run down the throat?", "Patient: I have been suffering from rhinitis for many years, which affects my sleep recently", "Patient: I used to take cephalosporin", "Patient: It will be relieved after taking it, but it will relapse slowly", " Doctor: Have you ever had a sinus CT scan and have sinusitis?", "Doctor: For the safety of your medication, do you have a history of drug allergies? Do you have poor liver and kidney function?", "Patient: Yes", "Patient: Sinusitis", "Patient: No allergies", "Patient: Normal liver and kidneys", "Patient: Only one of the two nostrils can pass through", "Patient: Pillow should be raised to ventilate", "Doctor: According to your description Considering the possibility of chronic sinusitis and chronic pharyngitis, you can consider [MASK], [MASK], [MASK], [MASK]"]}
dialogue3 = {"label": ["calamine lotion", "loratadine", "cetirizine", "glycyrrhizin"], "disease": ["dermatitis"], "dialog": [ "Patient: I have itchy red bumps all over my body, what should I do?", "Doctor: Hello, this is an allergy. How long has it been?", "Patient: Hello for three days", "Doctor: Before the rash , Have you eaten anything special?", "Patient: The chest and back are particularly heavy, and there are also on the legs", "Doctor: Have you taken any anti-allergic medicine?", "Patient: I have taken tuberculosis medicine, and the later cause Abnormal liver function, liver protection treatment for 5 days", "Doctor: How many days have you taken the tuberculosis medicine?", "Patient: Yiganling Capsule, Glucuronolide Tablet, Muyue Enzyme QIO Capsule and these medicines", "Doctor: How many days have you taken good tuberculosis medicine?", "Patient: For four months, the doctor asked to stop the liver function test first, and the liver function test of aspartate transanase 184. Let's stop the liver protection treatment, these few days I have a skin rash all over my body, what kind of medicine should I use? Thank you", "Doctor: Well, there is a high possibility of allergy to these hepatoprotective drugs", "Doctor: You can take [MASK, [MASK], [MASK] [MASK] orally. [MASK]"], "original_dialog": ["Patient: I have itchy red bumps all over my body, what should I do?", "Doctor: Hello, how long have you been allergic to?", "Patient: Hello It's been three days", "Doctor: Before the rash, did you take anything special?", "Patient: The chest and back are very large, and there are also on the legs", "Doctor: Have you taken any anti-allergic medicine?" , "Patient: Taking tuberculosis medicine, and then discontinued due to abnormal liver function, liver protection treatment for five days", "Doctor: How long did you take tuberculosis medicine?", "Patient: Yiganling capsule, glucurolactone tablet, Murayuki enzyme QIO capsules and these medicines", "Doctor: How many days have you taken good tuberculosis medicine?", "Patient: For four months, the doctor asked to stop the liver function test first, and the liver function test of aspartate transanase 184 .Let me stop the liver-protective treatment. I have a rash all over the body these days. What kind of medicine should I use? Thank you.", "Doctor: Okay, that is, there is a high possibility of allergy to these liver-protective drugs.", "Doctor: You can take [MASK, [MASK][MASK]Use [MASK]"]}
dialogue4 = {"label": ["Chuanbei Loquat Dew"], "disease": ["Upper respiratory tract infection"], "dialog": ["Patient: breastfeeding cough, sore throat, always want to spit, has taken three The sky is not getting better, what kind of medicine can I take?", "Doctor: Hello, you can use some sense of the four seasons or [MASK]"], "original_dialog": ["Patient: breastfeeding cough, sore throat, always I want to spit, but I have been taking Pudilan for three days and it has not improved, what kind of medicine can I take?", "Doctor: Hello, you can use some sense of the four seasons or [MASK]"]}
dialogue5 = {"label": ["erythromycin", "eucalyptus, lime and pinene enteric-coated soft capsules", "mometasone furoate", "cephalosporins"], "disease": ["rhinitis"], "dialog": [ "Patient: The rhinitis is on, the left nasal cavity is sore and painful, and the condition is more serious at night. What kind of medicine can I take orally", "Doctor: Hello, how long have you been in this situation?", "Patient: I had a cold a few days ago, which caused it, It's been two days", "Doctor: Is there any obvious nasal congestion, purulent nasal discharge?", "Patient: There is a lot of nasal discharge, nasal congestion, not purulent snot, transparent sticky snot. A little cold, no fever, left nasal cavity Soreness and pain", "Doctor: Have you ever had sinusitis before?", "Patient: Yes, it happens every year", "Doctor: It's a sinusitis in your case. Haven't been to Wuhan or been in contact with it. Someone from Wuhan, right?", "Patient: No no", "Patient: All at home?", "Patient: I have budesonide at home, what medicine can I take", "Doctor: If the pain is not particularly severe If so, you can take it orally first, [MASK], [MASK], [MASK], [MASK]"], "original_dialog": ["Patient: rhinitis attack, left nasal cavity soreness and pain, if it is more serious at night, you can take it orally What kind of medicine?", "Doctor: Hello, how long has this been the case?", "Patient: I have a cold a few days ago, which caused it, and it has been two days", "Doctor: Is there any obvious nasal congestion and runny nose? ", "Patient: Lots of snot, stuffy nose, not purulent snot, transparent viscous snot. A little cold, no fever, sore and painful left nasal cavity", "Doctor: Have you ever had sinusitis before?", "Patient: Yes, it happens every year", "Doctor: In your case, consider a sinusitis attack. Have you ever been to Wuhan or been in contact with people from Wuhan?", "Patient: No no", "Patient: All are there. At home?", "Patient: I have budesonide at home, what other medicines can I take", "Doctor: If the pain is not particularly severe, you can take [MASK], [MASK], [MASK], [MASK] orally first. ], [MASK]"]}

dialog = [dialogue1, dialogue2, dialogue3, dialogue4, dialogue5]

In [7]:
label_dict = {
  "berberine": 0,
  "Trimebutine": 1,
  "Probiotics": 2,
  "crack": 3,
  "dexamethasone": 4,
  "Ebastine": 5,
  "budesonide": 6,
  "amoxicillin": 7,
  "mometasone furoate": 8,
  "Fenghanmao Granules": 9,
  "omeprazole": 10,
  "Compound Digestive Enzymes": 11,
  "Mosapride": 12,
  "Aluminum-magnesium compound": 13,
  "Pudilan Anti-inflammatory Oral Liquid": 14,
  "Chuanbei Loquat Dew": 15,
  "erythromycin": 16,
  "Compound methoxyphenamine capsules": 17,
  "montelukast": 18,
  "Cold Spirit": 19,
  "Entecavir": 20,
  "glutathione": 21,
  "Su Huang Zhike Capsules": 22,
#   "Eucalyptus, Lime and Pinene Enteric Soft Capsules": 23, (Original)
  "Eucalyptus, Lime and Pinene Enteric-Coated Soft Capsules": 23,
  "Calamine Lotion": 24,
  "Tongqiao rhinitis granules": 25,
  "Lianhua Qingwen Capsule": 26,
  "oseltamivir": 27,
  "dextromethorphan": 28,
  "loratadine": 29,
  "Mesalazine": 30,
  "Tylenol": 31,
  "Silymarin": 32,
  "Polyene phosphatidylcholine": 33,
  "Physiological Sodium Chloride Solution": 34,
  "Montmorillonite": 35,
#     "Licorice Tablets": 36, (Original)
  "Licorice Tablet": 36,
  "minocycline": 37,
  "Sanjiuweitai": 38,
  "Hydrocortisone": 39,
  "Colloidal Bismuth Pectin": 40,
  "Glutamine": 41,
  "azithromycin": 42,
  "cephalosporins": 43,
  "adapalene": 44,
  "Ambroxol": 45,
  "tacrolimus": 46,
  "glycyrrhizin": 47,
  "Chang Yanning": 48,
  "levofloxacin": 49,
  "Panlangen": 50,
  "acetaminophen": 51,
  "cetirizine": 52,
  "Triamcinolone acetonide": 53,
  "Jianwei Xiaoshi Tablets": 54,
  "aciclovir": 55,
  "Ketoconazole": 56,
  "Anisodamine": 57,
  "ibuprofen": 58,
  "clarithromycin": 59,
  "Tenofovir": 60,
  "Metoclopramide": 61,
  "Hathinad": 62,
  "Sulphur soap": 63,
  "Shuanghuanglian Oral Liquid": 64,
  "antiviral particles": 65,
  "Bicyclic alcohol": 66,
  "New Healing Fluid": 67,
  "Selenium Disulfide": 68,
  "Fuzheng Huayu Capsule": 69
}

In [8]:
lower_case_label_dict = {}

for d in label_dict:
#     print(d,' ',label_dict[d])
    d_lower = d.lower()
    lower_case_label_dict [d_lower] = label_dict[d]
    
print(lower_case_label_dict)

{'berberine': 0, 'trimebutine': 1, 'probiotics': 2, 'crack': 3, 'dexamethasone': 4, 'ebastine': 5, 'budesonide': 6, 'amoxicillin': 7, 'mometasone furoate': 8, 'fenghanmao granules': 9, 'omeprazole': 10, 'compound digestive enzymes': 11, 'mosapride': 12, 'aluminum-magnesium compound': 13, 'pudilan anti-inflammatory oral liquid': 14, 'chuanbei loquat dew': 15, 'erythromycin': 16, 'compound methoxyphenamine capsules': 17, 'montelukast': 18, 'cold spirit': 19, 'entecavir': 20, 'glutathione': 21, 'su huang zhike capsules': 22, 'eucalyptus, lime and pinene enteric-coated soft capsules': 23, 'calamine lotion': 24, 'tongqiao rhinitis granules': 25, 'lianhua qingwen capsule': 26, 'oseltamivir': 27, 'dextromethorphan': 28, 'loratadine': 29, 'mesalazine': 30, 'tylenol': 31, 'silymarin': 32, 'polyene phosphatidylcholine': 33, 'physiological sodium chloride solution': 34, 'montmorillonite': 35, 'licorice tablet': 36, 'minocycline': 37, 'sanjiuweitai': 38, 'hydrocortisone': 39, 'colloidal bismuth pe

In [9]:
print(tokenizer.all_special_tokens)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [10]:
print(tokenizer.all_special_ids)

[100, 102, 0, 101, 103]


In [11]:
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]']
    }

tokenizer.add_special_tokens(p)

2

In [12]:
tokenizer.convert_tokens_to_ids('[DOC]')

30522

In [13]:
tokenizer.convert_tokens_to_ids('[PAT]')

30523

In [14]:
len(tokenizer)

30524

In [15]:
bert_model.resize_token_embeddings(len(tokenizer))

Embedding(30524, 768)

In [16]:
print(tokenizer.all_special_ids)

[100, 102, 0, 101, 103, 30522, 30523]


In [17]:
print(tokenizer.all_special_tokens)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]']


In [18]:
def replacement(dlog):
    dialog_list = []
    whos_turn = []
    
    for d in dlog:
        dialog = d.lower()
        idx_pat = re.search('patient', dialog)
        idx_doc = re.search('doctor', dialog)
        
        if(idx_pat):
            z = re.sub('patient', '[PAT]', dialog)
            whos_turn.append('p')
        
        else:
            z = re.sub('doctor', '[DOC]', dialog)
            whos_turn.append('d')
        
        dialog_list.append(z)
   
    return dialog_list, whos_turn     

In [19]:
def find_block(whos_turn):
    block = {}
    cnt = 0
    
    for i in range(len(whos_turn)):
        if i==0:
            block[i]=cnt
        elif whos_turn[i] == whos_turn[i-1]:
             block[i]=cnt
        elif whos_turn[i] != whos_turn[i-1]:
                   block[i] = cnt+1
                   cnt = cnt+1
                   
    return block               

In [20]:
def make_edges(whos_turn, block):
    edge_list = []
    
    for i in range(len(whos_turn)):
        for j in range(i+1, len(whos_turn)):
            if((block[i] == block[j]) or block[i] + 1 == block[j]):
                edge_1 = [i,j]
                edge_2 = [j,i]
                edge_list.append(edge_1)
                edge_list.append(edge_2)
    
    edge_list = torch.tensor(edge_list, dtype = torch.long)
    edge_list = edge_list.T
    
    return edge_list

In [21]:
def labels_list_generate(labels):
    labels_list = []
# labels = [1,2,3]
# 0 0
# 1 0
# 1 1
# 0 1
# labels_one_hot = [0,1,1,1,rest_zero]
#     one_hot_label_list = []

#     for i in range(70):
#         one_hot_label_list.append(0)
# #     print(one_hot_label_list)
    
#     one_hot_label_list = torch.Tensor(one_hot_label_list)
    one_hot_label_list = torch.zeros(1,70)

    for label in labels:
        label_lower = label.lower()
#         print(label_lower)
#         print(lower_case_label_dict[label_lower])
        number = lower_case_label_dict[label_lower]
        one_hot_label_list[0][number] = 1
#         print(one_hot_label_list)
#         print('error above')
#         labels_list.append(lower_case_label_dict[label_lower])
    
#     labels_list = torch.Tensor(labels_list)
#     print(one_hot_label_list.shape)
#     torch.reshape(one_hot_label_list, (1,-1))
    print(one_hot_label_list.shape)
    return one_hot_label_list

In [22]:
data_list = []
for dg in dialog:
    dlog = dg['dialog']
    
    dlog_list, whos_turn = replacement(dlog)
    
    encoding_dlog = tokenizer(dlog_list, return_tensors = 'pt', padding = True)
    
    output_encoding = bert_model(**encoding_dlog)
    
    utterance_encoding = output_encoding['last_hidden_state'].permute(1,0,2)[0]
    
    block = find_block(whos_turn)
    
    edge_list = make_edges(whos_turn, block)
    print(edge_list.dtype)
    edge_weights = torch.tensor([1] * edge_list.shape[1], dtype = torch.float32)
    
    label_dlog = dg['label']
    
    one_hot_label_list = labels_list_generate(label_dlog)
    
    data_train = Data(x = utterance_encoding, edge_index = edge_list, edge_attr = edge_weights, y = one_hot_label_list)
    
    data_list.append(data_train)
    
    
    

torch.int64
torch.Size([1, 70])
torch.int64
torch.Size([1, 70])
torch.int64
torch.Size([1, 70])
torch.int64
torch.Size([1, 70])
torch.int64
torch.Size([1, 70])


In [23]:
data_list

[Data(x=[9, 768], edge_index=[2, 48], edge_attr=[48], y=[1, 70]),
 Data(x=[22, 768], edge_index=[2, 168], edge_attr=[168], y=[1, 70]),
 Data(x=[13, 768], edge_index=[2, 26], edge_attr=[26], y=[1, 70]),
 Data(x=[2, 768], edge_index=[2, 2], edge_attr=[2], y=[1, 70]),
 Data(x=[12, 768], edge_index=[2, 32], edge_attr=[32], y=[1, 70])]

In [24]:
class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
    
    def process(self):
#         # Read data into huge `Data` list.
        data_list1 = data_list

        if self.pre_filter is not None:
            data_list1 = [data for data in data_list1 if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list1 = [self.pre_transform(data) for data in data_list1]

        data, slices = self.collate(data_list1)
        torch.save((data, slices), self.processed_paths[0])

In [25]:
dataset = MyOwnDataset(root="my_own_dataset/")

In [26]:
my_own = MyOwnDataset(root="my_own_dataset/")
my_own

MyOwnDataset(5)

In [27]:
my_own[0].num_features

768

In [28]:
my_own.num_classes

70

In [29]:
my_own[0].num_nodes

9

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [31]:
train_loader = DataLoader(data_list, batch_size=2)
train_loader

/Users/mohit/opt/anaconda3/envs/exp_pytorch_only/lib/python3.7/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [32]:
for step, data in enumerate(train_loader):
    print('Step: ', step + 1)
    print()
    print('Number of graphs in current batch : ', data.num_graphs)
    print(data)
    print('Number of classes shape : ',data.y.shape)
#     print('Number of classes : ',data.y)
#     print('Data batch : ', data.batch.shape[0])
    print('\n')

Step:  1

Number of graphs in current batch :  2
DataBatch(x=[31, 768], edge_index=[2, 216], edge_attr=[216], y=[2, 70], batch=[31], ptr=[3])
Number of classes shape :  torch.Size([2, 70])


Step:  2

Number of graphs in current batch :  2
DataBatch(x=[15, 768], edge_index=[2, 28], edge_attr=[28], y=[2, 70], batch=[15], ptr=[3])
Number of classes shape :  torch.Size([2, 70])


Step:  3

Number of graphs in current batch :  1
DataBatch(x=[12, 768], edge_index=[2, 32], edge_attr=[32], y=[1, 70], batch=[12], ptr=[2])
Number of classes shape :  torch.Size([1, 70])




In [33]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args):
        super(GAT,self).__init__()
        
        self.conv1 = GATConv(input_dim, hidden_dim, heads=args['heads'])
        self.conv2 = GATConv(hidden_dim*args['heads'], hidden_dim, heads = args['heads'])
        
        self.lin = torch.nn.Linear(hidden_dim*args['heads'], output_dim)
        
    def forward(self, data, batch):
        x, edge_index = data.x, data.edge_index
        
        x=self.conv1(x, edge_index)
        x=F.dropout(F.relu(x), p=args['dropout'], training=self.training)
        
        x=self.conv2(x, edge_index)
        
        
        x=global_mean_pool(x, batch)
        
        x=F.dropout(F.relu(x), p=args['dropout'], training=self.training)
        
        x = self.lin(x)
        
        x = F.sigmoid(x)
        
#         x = x.view(-1)
#         x = torch.nn.Sigmoid(x)
        return x
        

In [34]:

# class GAT(torch.nn.Module):
#     def __init__(self,input_dim, hidden_dim, output_dim,args):
#         super(GAT, self).__init__()
#         #use our gat message passing 
#         self.conv1 = GATConv(input_dim, hidden_dim, heads=args['heads'])
#         self.conv2 = GATConv(args['heads'] * hidden_dim, hidden_dim, heads=args['heads'])
        
#         self.post_mp = torch.nn.Sequential(
#             torch.nn.Linear(args['heads'] * hidden_dim, hidden_dim), torch.nn.Dropout(args['dropout'] ), 
#             torch.nn.Linear(hidden_dim, output_dim))
        
#     def forward(self, data, batch):
#         x, edge_index = data.x, data.edge_index
#         # Layer 1
#         x = self.conv1(x, edge_index)
#         x = F.dropout(F.relu(x), p=args['dropout'], training=self.training)
#         # Layer 2
#         x = self.conv2(x, edge_index)
#         x = F.dropout(F.relu(x), p=args['dropout'], training=self.training)
#         # MLP output
#         x = self.post_mp(x)
        
#         x=global_mean_pool(x, batch)
        
#         return F.sigmoid(x)
       

In [35]:
args={
      'heads':2,
      'dropout': 0.5
      }

In [36]:
model = GAT(input_dim = 768, hidden_dim = 64, output_dim = 70, args = args)
model

GAT(
  (conv1): GATConv(768, 64, heads=2)
  (conv2): GATConv(128, 64, heads=2)
  (lin): Linear(in_features=128, out_features=70, bias=True)
)

In [37]:
model.float().to(device)

GAT(
  (conv1): GATConv(768, 64, heads=2)
  (conv2): GATConv(128, 64, heads=2)
  (lin): Linear(in_features=128, out_features=70, bias=True)
)

In [41]:
type(model.parameters())


generator

In [62]:
# train_loader = train_loader.to(device)
# for data in train_loader:
#     print(data)
#     data = data.to(device)

In [63]:

# print('Batch size : ',data.batch)

In [64]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    
#     hidden = model.init_hidden(140)
    
    for data in train_loader:
        
#         optimizer.zero_grad()
# #         output = model(x)
#         output = model(data, data.batch)
#         print('output shape : ', output.shape)
#         print('output : ',output)
# #         loss = criterion(output, y)
#         loss = criterion(output, data.y)
#         loss.backward(retain_graph=True)
# #         loss.backward()
#         optimizer.step()
#         print('Loss: {:.3f}'.format(loss.item()))
        
        
        
        
#         hidden = repackage_hidden(hidden)
           
        optimizer.zero_grad()
#         print(data.batch)
        out = model(data, data.batch)
#         out = torch.randn(data.num_graphs, 70)
        print('Prediction : ', out)
        print('Prediction shape : ', out.shape)
        print('Ground Truth Shape : ',data.y.shape)
        print('Ground Truth : ',data.y)
        
        loss = criterion(out, data.y)
        print('loss : ',loss)
#         loss.requires_grad = True
        loss.backward(retain_graph=True) 
        
        
#         loss.backward()
        optimizer.step()
    
    



In [65]:
# def train():
#     model.train()
# #     criterion = torch.nn.BCEWithLogitsLoss()
    
#     optimizer.zero_grad()
#     out = model(data, data.batch)
#     loss = criterion(out, data.y.to(torch.float))
#     loss.backward(retain_graph=True)
#     optimizer.step()

#     return loss.item()

In [66]:
# def test(loader):
#     model.eval()
    
#     correct = 0
#     for data in loader:
#         out = model(data, data.batch)
#         print('Batch length : ', data.batch.shape[0])
#         print('Size of output : ', out.shape)
        
            

In [67]:
for epoch in range(3):
    print('Epoch number : ', epoch+1)
    train()
#     test(train_loader)

Epoch number :  1
Prediction :  tensor([[0.5140, 0.4651, 0.4673, 0.4803, 0.5188, 0.4743, 0.4722, 0.4560, 0.4802,
         0.5712, 0.4709, 0.5455, 0.4157, 0.5079, 0.5212, 0.4914, 0.5004, 0.5322,
         0.4877, 0.3956, 0.4137, 0.4929, 0.4049, 0.5735, 0.5077, 0.5134, 0.4738,
         0.5134, 0.4516, 0.5330, 0.4874, 0.4877, 0.5069, 0.4485, 0.4816, 0.5004,
         0.4554, 0.4197, 0.4550, 0.5122, 0.4735, 0.4893, 0.4958, 0.4690, 0.5002,
         0.5700, 0.4223, 0.5011, 0.4649, 0.5817, 0.5440, 0.4450, 0.3826, 0.3820,
         0.5485, 0.5041, 0.5037, 0.4179, 0.4872, 0.5684, 0.4571, 0.4680, 0.4574,
         0.4601, 0.4815, 0.4826, 0.4473, 0.5235, 0.5274, 0.4849],
        [0.5308, 0.5181, 0.4803, 0.4251, 0.4764, 0.4163, 0.5028, 0.4576, 0.4931,
         0.5653, 0.5211, 0.5621, 0.3849, 0.5462, 0.5144, 0.5026, 0.5200, 0.5027,
         0.5071, 0.3818, 0.4037, 0.5134, 0.4042, 0.4953, 0.5542, 0.4977, 0.5108,
         0.4807, 0.4688, 0.4858, 0.5405, 0.5167, 0.4423, 0.5120, 0.4982, 0.5254,
         0.

/Users/mohit/opt/anaconda3/envs/exp_pytorch_only/lib/python3.7/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Prediction :  tensor([[0.4805, 0.2536, 0.1962, 0.2109, 0.5300, 0.4745, 0.5942, 0.4498, 0.3167,
         0.3801, 0.6701, 0.4815, 0.2883, 0.3833, 0.4002, 0.3539, 0.4637, 0.4861,
         0.3130, 0.1316, 0.4364, 0.2712, 0.2048, 0.4562, 0.4169, 0.3009, 0.5450,
         0.2747, 0.4019, 0.4221, 0.3515, 0.4104, 0.3915, 0.2605, 0.5989, 0.3294,
         0.5510, 0.3003, 0.4086, 0.4045, 0.3312, 0.5750, 0.3883, 0.1246, 0.4986,
         0.5658, 0.1929, 0.6749, 0.2615, 0.6207, 0.5497, 0.5370, 0.4058, 0.3813,
         0.5040, 0.5187, 0.5393, 0.3945, 0.4391, 0.6494, 0.3841, 0.5056, 0.5003,
         0.2215, 0.3705, 0.3645, 0.6091, 0.2102, 0.4688, 0.4319],
        [0.1810, 0.1812, 0.0564, 0.3938, 0.1450, 0.2634, 0.1860, 0.5314, 0.3430,
         0.4353, 0.3335, 0.3225, 0.0189, 0.3793, 0.2808, 0.0861, 0.6491, 0.1996,
         0.5985, 0.0259, 0.0980, 0.2420, 0.1722, 0.1969, 0.5806, 0.2712, 0.4160,
         0.0991, 0.2086, 0.5173, 0.2168, 0.3760, 0.2106, 0.1247, 0.3073, 0.1090,
         0.3715, 0.0290, 0.05

Prediction :  tensor([[6.0132e-09, 1.0587e-15, 8.1854e-14, 2.6627e-14, 2.2443e-10, 8.7240e-14,
         1.0781e-05, 1.4088e-05, 1.0342e-02, 9.7916e-09, 3.5488e-09, 9.9735e-07,
         1.2274e-15, 1.5801e-09, 1.7502e-05, 2.0085e-10, 1.0647e-10, 8.2166e-09,
         6.1938e-10, 2.3559e-15, 8.3240e-12, 5.3658e-11, 3.1189e-15, 2.6884e-09,
         2.5561e-12, 1.1021e-14, 4.0450e-09, 5.1173e-13, 4.3400e-09, 5.8545e-11,
         8.3924e-13, 1.4484e-08, 1.9948e-11, 2.4919e-08, 1.9879e-12, 4.6580e-12,
         4.6598e-06, 1.7954e-12, 2.5500e-16, 5.7165e-11, 2.5032e-08, 4.6614e-09,
         1.9682e-10, 6.6634e-08, 2.3779e-10, 6.5096e-05, 6.7712e-17, 3.7126e-08,
         4.6948e-15, 2.6627e-10, 4.1108e-07, 5.8460e-10, 1.0963e-12, 6.9167e-14,
         1.9199e-09, 3.4672e-09, 6.2215e-09, 2.9375e-15, 1.7254e-07, 1.1642e-04,
         1.8555e-09, 5.9203e-14, 9.8143e-15, 5.8520e-13, 1.9164e-10, 1.0236e-12,
         3.1103e-13, 8.9796e-13, 4.0478e-17, 8.6770e-11]],
       grad_fn=<SigmoidBackward0>)
P

In [55]:
# model = torch.nn.Linear(20, 5) # predict logits for 5 classes
# x = torch.randn(1, 20)

# y = torch.tensor([[1., 0., 1., 0., 0.]]) # get classA and classC as active

# criterion = torch.nn.BCEWithLogitsLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# for epoch in range(20):
#     optimizer.zero_grad()
#     output = model(x)
#     print('output shape : ', output.shape)
#     print('output : ',output)
#     loss = criterion(output, y)
#     loss.backward()
#     optimizer.step()
#     print('Loss: {:.3f}'.format(loss.item()))

In [78]:
def test(loader):
    model.eval()
    
    correct = 0
    for data in loader:
        out = model(data, data.batch)
        print(out.shape)
        print(out)
#         print(out>=0.5)
        z = out>=0.5
        print(z)
        z=z.float()
        print('z conversion',z)
#         correct +=
        print('================')
        print(data.y.dtype)
        print()

In [79]:
test(train_loader)

torch.Size([2, 70])
tensor([[2.0265e-22, 2.9333e-24, 8.6556e-21, 1.2679e-25, 1.5628e-19, 4.0780e-20,
         6.2552e-19, 7.3928e-19, 5.4823e-03, 2.1981e-24, 8.1098e-18, 1.9284e-20,
         3.5448e-21, 6.9839e-19, 6.1903e-14, 2.6594e-14, 1.5414e-18, 1.4704e-21,
         3.1492e-18, 2.8240e-19, 5.6156e-24, 2.2091e-19, 7.9051e-23, 4.2616e-18,
         3.0735e-19, 1.4181e-22, 1.7397e-21, 5.3294e-19, 1.6275e-19, 1.0479e-15,
         5.3204e-20, 2.1663e-22, 9.6315e-22, 6.5668e-19, 2.2329e-15, 4.1466e-20,
         1.6481e-16, 1.5697e-18, 5.9706e-17, 3.6017e-21, 7.8032e-18, 2.7026e-19,
         2.1188e-19, 1.1113e-14, 9.7257e-19, 5.0408e-20, 2.0716e-19, 3.5359e-15,
         4.5073e-17, 7.4046e-17, 1.4968e-22, 1.5408e-20, 3.1059e-14, 2.3502e-20,
         5.8996e-24, 3.6361e-20, 2.1251e-18, 6.3096e-16, 1.1078e-20, 1.3728e-09,
         1.1722e-25, 1.1577e-22, 6.2686e-20, 3.3812e-19, 2.9471e-20, 1.3221e-22,
         9.1905e-20, 1.4901e-20, 6.1287e-23, 1.7323e-24],
        [1.6238e-21, 2.5789e-23